In [1]:
import json
from datetime import datetime

from my_tsne import TrajectoryTSNE
from plotter import TrajectoryPlotter
from trajectory import DataTrajectory, TopologyConverter
from analyse import MultiTrajectory, SingleTrajectory
from utils.param_key import *

# Define the file names and paths for your data
1. trajectory_name = Name of the trajectory (used for plot titles)
2.  filename_list = The list of filenames of the trajectories (e.g. .xtc/.dcd- files)
3. topology_filename: .pdb-file of the trajectory
4. folder_path: path to the folder, where the trajectory files are (make sure to have the pdb data and the trajectories in the same folder)

In [2]:
def get_files_and_kwargs(params):
    trajectory_name = params[TRAJECTORY_NAME]
    file_element = params[FILE_ELEMENT]
    if trajectory_name == '2f4k':
        filename_list = [f'2F4K-0-protein-{i:03d}.dcd' for i in range(0, 62 + 1)] + ['tr3_unfolded.xtc', 'tr8_folded.xtc']
        kwargs = {'filename': filename_list[file_element],
                  'topology_filename': '2f4k.pdb',
                  'folder_path': 'data/2f4k',
                  'params': params}
    elif trajectory_name == 'prot2':
        filename_list = ['prod_r1_nojump_prot.xtc',
                         'prod_r2_nojump_prot.xtc',
                         'prod_r3_nojump_prot.xtc']
        kwargs = {'filename': filename_list[file_element],
                  'topology_filename': 'prod_r1_pbc_fit_prot_last.pdb',
                  'folder_path': 'data/ProtNo2',
                  'params': params}
    elif trajectory_name == 'savinase':
        filename_list = ['savinase_1.xtc', 'savinase_2.xtc']
        kwargs = {'filename': filename_list[file_element],
                  'topology_filename': 'savinase.pdb',
                  'folder_path': 'data/Savinase',
                  'params': params}
    elif trajectory_name == '2wav':
        filename_list = [f'2WAV-0-protein-{i:03d}.dcd' for i in range(0, 136)]
        kwargs = {'filename': filename_list[file_element],
                  'topology_filename': '2wav.pdb',
                  'folder_path': 'data/2WAV-0-protein',
                  'params': params, 'atoms': list(range(710))}
    elif trajectory_name == '5i6x':
        filename_list = ['protein.xtc', 'system.xtc']
        kwargs = {'filename': filename_list[file_element],
                  'topology_filename': '5i6x.pdb',
                  'folder_path': 'data/ser-tr',
                  'params': params}
    else:
        raise ValueError(f'No data trajectory was found with the name `{trajectory_name}`.')
    filename_list.pop(file_element)
    return filename_list, kwargs

# Initialize Model Parameters (If the json file is not used)
### Old Class-algorithms implementations with parameters, not strings (deprecated):
USE_STD: True, CENTER_OVER_TIME: False (only for tensor),
{ALGORITHM_NAME: 'pca', NDIM: TENSOR_NDIM, USE_STD: True, CENTER_OVER_TIME: False}

## Original Algorithms
### PCA
1. {ALGORITHM_NAME: 'original_pca', NDIM: MATRIX_NDIM} or
2. {ALGORITHM_NAME: 'pca', NDIM: TENSOR_NDIM, USE_STD: False, ABS_EVAL_SORT: False}
### TICA
1. {ALGORITHM_NAME: 'original_tica', NDIM: MATRIX_NDIM} or
2. {ALGORITHM_NAME: 'tica', LAG_TIME: params[LAG_TIME], NDIM: MATRIX_NDIM, USE_STD: False, ABS_EVAL_SORT: False}

### raw MATRIX models
{ALGORITHM_NAME: 'pca', NDIM: MATRIX_NDIM},
{ALGORITHM_NAME: 'tica', NDIM: MATRIX_NDIM, LAG_TIME: params[LAG_TIME]},

### raw TENSOR models
{ALGORITHM_NAME: 'pca', NDIM: TENSOR_NDIM}
{ALGORITHM_NAME: 'tica', NDIM: TENSOR_NDIM, LAG_TIME: params[LAG_TIME]}

## Parameters
### KERNEL
KERNEL_ONLY, KERNEL_DIFFERENCE, KERNEL_MULTIPLICATION
### KERNEL_TYPE
MY_GAUSSIAN, MY_EXPONENTIAL, MY_LINEAR, MY_EPANECHNIKOV, (GAUSSIAN, EXPONENTIAL, ...)
### COV_FUNCTION
 np.cov, np.corrcoef, utils.matrix_tools.co_mad
### NTH_EIGENVECTOR (int)
### LAG_TIME (int)
### Boolean Parameters
CORR_KERNEL, ONES_ON_KERNEL_DIAG, USE_STD, CENTER_OVER_TIME, EXTRA_DR_LAYER

In [3]:
def get_model_params_list(alg_json_file, params):
    if alg_json_file is not None:
        return json.load(open(alg_json_file))
        # return json.load(open('algorithm_parameters_list.json'))
    else:
        return [
            # Original Algorithms
            {ALGORITHM_NAME: 'original_pca', NDIM: MATRIX_NDIM},
            {ALGORITHM_NAME: 'original_tica', NDIM: MATRIX_NDIM},
            # Insert your model parameters
            {ALGORITHM_NAME: 'pca', NDIM: TENSOR_NDIM, KERNEL: KERNEL_ONLY, ANALYSE_PLOT_TYPE: PLOT_3D_MAP},
        ]

# Define the parameter grid, for a model grid search

In [4]:
def get_param_grid():
    param_grid = [
        {
            ALGORITHM_NAME: ['pca', 'tica'],
            KERNEL: [None],
        }, {
            ALGORITHM_NAME: ['pca', 'tica', 'kica'],
            LAG_TIME: [10],
            KERNEL: [KERNEL_DIFFERENCE, KERNEL_MULTIPLICATION, KERNEL_ONLY],
            KERNEL_TYPE: [MY_LINEAR, MY_GAUSSIAN, MY_EXPONENTIAL, MY_EPANECHNIKOV],
            ONES_ON_KERNEL_DIAG: [True, False],
            # EXTRA_DR_LAYER: [False, True],
            # EXTRA_LAYER_ON_PROJECTION: [False, True],
            ABS_EVAL_SORT: [False, True]
        }
    ]
    return param_grid

# Define parameters for different runs

In [5]:
run_params_json = None  # NotYetImplemented
# alg_params_json = 'config_files/algorithm/pca+gaussian_kernels.json'  # None or filename
alg_params_json = None  # 'config_files/algorithm/pca+gaussian_kernels.json'  # None or filename
# alg_params_json = 'config_files/algorithm/pca+gaussian_kernels_with_2nd_layer.json'
# alg_params_json = 'config_files/algorithm/pca+tica+all_kernels.json'  # None or filename
run_params = {
    PLOT_TYPE: COLOR_MAP,  # 'heat_map', 'color_map', '3d_map', 'explained_var_plot'
    PLOT_TICS: True,  # True, False
    STANDARDIZED_PLOT: False,  # True, False
    CARBON_ATOMS_ONLY: True,  # True, False
    INTERACTIVE: True,  # True, False
    N_COMPONENTS: 2,
    LAG_TIME: 10,
    TRUNCATION_VALUE: 0,  # deprecated
    BASIS_TRANSFORMATION: False,
    USE_ANGLES: False,
    TRAJECTORY_NAME: '2f4k',
    FILE_ELEMENT: 0,
}

filename_list, kwargs = get_files_and_kwargs(run_params)
model_params_list = get_model_params_list(alg_params_json, run_params)
param_grid = get_param_grid()

# Compare models (Qualitative)
Plots the different models side by side, demonstrating the reduced dimensions

Note: qualitative plot works only for 2 components

In [6]:
if run_params[N_COMPONENTS] != 2:
    raise ValueError("Qualitative plot only works for 2 Components")
else:
    tr = DataTrajectory(**kwargs)
    SingleTrajectory(tr).compare(model_params_list)


Loading trajectory 2F4K-0-protein-000.dcd...


C:\Users\poliv\.conda\envs\tSNEpy\lib\site-packages\mdtraj\formats\pdb\pdbstructure.py:435: UserWarning: WARNING: two consecutive residues with same number (ATOM      9  N   SER X   1      21.350  12.200  22.900  1.00  0.00      SYST    , ATOM      8  O   LEU X   1      20.450  10.170  22.620  1.00  0.00      SYST    )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))
C:\Users\poliv\.conda\envs\tSNEpy\lib\site-packages\mdtraj\formats\pdb\pdbstructure.py:435: UserWarning: WARNING: two consecutive residues with same number (ATOM     15  N   ASP X   1      19.120  15.080  23.360  1.00  0.00      SYST    , ATOM     14  OG  SER X   1      21.420  13.550  25.410  1.00  0.00      SYST    )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))
C:\Users\poliv\.conda\envs\tSNEpy\lib\site-packages\mdtraj\formats\pdb\pdbstructure.py:435: UserWarning: WARNING: tw

Trajectory `<mdtraj.Trajectory with 10000 frames, 577 atoms, 68 residues, and unitcells>` successfully loaded.
Running {'algorithm_name': 'original_pca', 'ndim': 2}...
Running {'algorithm_name': 'original_tica', 'ndim': 2}...
Running {'algorithm_name': 'pca', 'ndim': 3, 'kernel': 'only', 'analyze_plot_type': '3d_map'}...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


(10000, 2)
EV: [[ 0.26239878 -0.06129316  0.12518095 ...  0.00252795  0.00742408
  -0.00605256]
 [ 0.02527856 -0.08266236 -0.20172598 ... -0.02799846  0.00441074
   0.01981123]
 [ 0.07293204  0.37324318  0.01126772 ...  0.00246817 -0.02506632
  -0.00666858]
 ...
 [-0.25745672  0.07039433 -0.01809802 ... -0.01151257 -0.00314898
   0.00179545]
 [-0.08461455  0.06773924 -0.12456944 ... -0.01051262 -0.00826963
   0.00899039]
 [-0.06493025 -0.08180034  0.27743328 ... -0.00567892 -0.00088377
   0.00220037]] EW: [3.72068846e+00 2.74853652e+00 1.57697940e+00 1.15451429e+00
 8.83082404e-01 7.45085822e-01 6.83097351e-01 4.92029494e-01
 3.84907412e-01 3.46247976e-01 3.02984864e-01 2.11876526e-01
 1.93783854e-01 1.50039841e-01 1.39670760e-01 1.26935658e-01
 1.10887976e-01 9.95413738e-02 8.60489921e-02 8.01323967e-02
 7.75909483e-02 6.87578838e-02 6.54801968e-02 5.76376689e-02
 5.42604402e-02 5.21125518e-02 4.42887458e-02 4.27071192e-02
 3.87699483e-02 3.61800106e-02 3.50843849e-02 3.20297746e-02
 